#### Define Imports

In [9]:
%pip install openai==0.28

  Attempting uninstall: openai
    Found existing installation: openai 1.35.7
    Uninstalling openai-1.35.7:
      Successfully uninstalled openai-1.35.7
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\python\python.exe -m pip install --upgrade pip' command.


In [13]:
import requests
import pandas as pd 
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import json
import openai
import os

from environment import env

config = env.env()
api_key = config['api_key']

api_base_url = 'https://developer.nps.gov/api/v1/'


#### Define Functions

In [7]:
def handle_query(query, use_intent=True):
    prompt = f"Query: {query}\n"
    if use_intent:
        prompt += f"Intent: {intent}\n"
    response = openai.Completion.create(
        model=fine_tune['fine_tuned_model'],  # Use the fine-tuned model
        prompt=prompt,
        max_tokens=50
    )
    completion = response['choices'][0]['text'].strip()
    return completion



#### Define Variables

#### Read Inputs

In [14]:
synthetic_queries_df = pd.read_csv('synthetic_queries.csv')

# Upload and prepare the training file
train_file = openai.File.create(file=open('fine_tune_train_data.jsonl'), purpose='fine-tune')
val_file = openai.File.create(file=open('fine_tune_val_data.jsonl'), purpose='fine-tune')

# Check if the files are prepared
train_file_id = train_file['id']
val_file_id = val_file['id']

APIRemovedInV1: 

You tried to access openai.File, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
fine_tune = openai.FineTune.create(
    training_file=train_file_id,
    validation_file=val_file_id,
    model="davinci"  # or "curie" for a smaller model
)

# Monitor the fine-tuning job
fine_tune_id = fine_tune['id']

# Wait for fine-tuning to complete
status = openai.FineTune.retrieve(fine_tune_id)['status']
while status not in ['succeeded', 'failed']:
    fine_tune = openai.FineTune.retrieve(fine_tune_id)
    status = fine_tune['status']
    print(f"Status: {status}")
    if status == 'succeeded':
        print("Fine-tuning succeeded")
    elif status == 'failed':
        print("Fine-tuning failed")
    else:
        import time
        time.sleep(60)  # Wait for 1 minute before checking again

In [ ]:
# Example usage
query = "What is the info for park YS?"
api_call = handle_query(query)
print(api_call)